# Predicción del Valor de Mercado de Coches Usados 🚗💰  

En este proyecto, desarrollaremos un modelo de machine learning para predecir el valor de mercado de coches usados a partir de sus características técnicas. Utilizaremos varias técnicas de modelado y compararemos su rendimiento.  
El proyecto está diseñado para optimizar:  
- La precisión de las predicciones  
- La velocidad del modelo  
- El tiempo de entrenamiento  

## Descripción del conjunto de datos 📊  
El dataset contiene información sobre coches usados, incluyendo características como:  
- **VehicleType**: tipo de carrocería del vehículo  
- **RegistrationYear**: año de matriculación  
- **Power**: potencia del vehículo en caballos de fuerza (CV)  
- **Mileage**: kilometraje acumulado en kilómetros  
- **Brand**: marca del vehículo  
- **FuelType**: tipo de combustible  

El objetivo es predecir la variable `Price` (precio del coche en euros) utilizando diferentes modelos.  

## Metodología ⚙️  
1. **Carga y exploración del dataset:** Identificación de valores faltantes, tipos de datos y estructura.  
2. **Preprocesamiento de datos:** Limpieza y transformación de variables categóricas y numéricas.  
3. **Entrenamiento de modelos:** Compararemos los siguientes modelos debido a su capacidad de manejar diferentes complejidades en los datos:
- Regresión lineal: Para establecer una base de comparación.
- Random Forest: Un modelo basado en árboles de decisión que captura relaciones no lineales.
- LightGBM: Un modelo de potenciación del gradiente eficiente en grandes conjuntos de datos.

4. **Evaluación del rendimiento:** Utilizaremos la métrica RMSE (Root Mean Squared Error) para medir la precisión de las predicciones.  

## Importación de librerías  
Primero, importamos las librerías necesarias para el análisis y modelado.

# Importación de librerías 🛠️  
En este paso, importamos todas las librerías necesarias para la manipulación de datos, el preprocesamiento y el entrenamiento de los modelos de machine learning.

In [20]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Carga y exploración inicial del dataset 📊  
Leemos el archivo CSV que contiene los datos de los coches usados y realizamos un examen inicial de su estructura.  
Utilizamos las funciones `head()`, `info()` y `describe()` para entender la cantidad de valores faltantes, los tipos de datos y obtener estadísticas básicas.

In [21]:
df = pd.read_csv('car_data.csv')
print(df.head())
df.info()

        DateCrawled  Price VehicleType  RegistrationYear Gearbox  Power  \
0  24/03/2016 11:52    480         NaN              1993  manual      0   
1  24/03/2016 10:58  18300       coupe              2011  manual    190   
2  14/03/2016 12:52   9800         suv              2004    auto    163   
3  17/03/2016 16:54   1500       small              2001  manual     75   
4  31/03/2016 17:25   3600       small              2008  manual     69   

   Model  Mileage  RegistrationMonth  FuelType       Brand NotRepaired  \
0   golf   150000                  0    petrol  volkswagen         NaN   
1    NaN   125000                  5  gasoline        audi         yes   
2  grand   125000                  8  gasoline        jeep         NaN   
3   golf   150000                  6    petrol  volkswagen          no   
4  fabia    90000                  7  gasoline       skoda          no   

        DateCreated  NumberOfPictures  PostalCode          LastSeen  
0  24/03/2016 00:00               

# Limpieza y preprocesamiento de datos 🧹  
En esta etapa, manejamos los valores faltantes y eliminamos columnas irrelevantes. También convertimos variables categóricas en un formato adecuado para el modelado.

Rellenamos los valores faltantes con el valor más apropiado según el contexto:  
- `NotRepaired`: Los vehículos sin reparación se rellenan como `no`.  
- `VehicleType`: Se rellena con `unknown` en caso de faltar información.  
- `Gearbox`: Se asume `manual` si falta el tipo de caja de cambios.  
- `Model`: Se rellena con el modelo más frecuente.


Eliminamos columnas que no aportan información relevante para la predicción del precio:  
- `DateCrawled`, `DateCreated`, `PostalCode`, `LastSeen` y `NumberOfPictures`.

Convertimos las columnas categóricas en el formato `category` para optimizar la memoria y facilitar el preprocesamiento.

In [22]:
# Rellenar valores faltantes con 'unknown' o el más frecuente para 'Model'
df['NotRepaired'].fillna('no', inplace=True)
df['VehicleType'].fillna('unknown', inplace=True)
df['Gearbox'].fillna('manual', inplace=True)
df['Model'].fillna(df['Model'].mode()[0], inplace=True)
df['FuelType'].fillna('petrol', inplace=True)

# Eliminar columnas que no son necesarias o tienen información redundante
df.drop(['DateCrawled', 'DateCreated', 'PostalCode', 'LastSeen', 'NumberOfPictures'], axis=1, inplace=True)

# Convertir a categóricas las columnas que tienen tipos de datos como 'object'
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].astype('category')

<ipython-input-22-b3fbd0c2bea7>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['NotRepaired'].fillna('no', inplace=True)
<ipython-input-22-b3fbd0c2bea7>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

# Entrenamiento y Evaluación del Modelo de Machine Learning 🔍

## División de los datos 📊  
Dividimos el conjunto de datos en dos partes:  
- **Conjunto de entrenamiento (80%)**: Para entrenar el modelo.  
- **Conjunto de prueba (20%)**: Para evaluar el rendimiento del modelo y medir su precisión.

In [23]:
# Dividir los datos
X = df.drop('Price', axis=1)
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preparar un pipeline de preprocesamiento para variables numéricas y categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['category']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

#Entrenamiento de los modelos 📈
##Entrenamos y comparamos tres modelos:

Regresión lineal: Para establecer una base inicial.
Random Forest: Un modelo basado en árboles de decisión.
LightGBM: Un modelo de potenciación del gradiente optimizado para grandes conjuntos de datos.

In [24]:
# Entrenamiento de regresión lineal
lr_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', LinearRegression())])
lr_model.fit(X_train, y_train)

# Entrenamiento de Random Forest
rf_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42))])
rf_model.fit(X_train, y_train)

# Entrenamiento de LightGBM
lgbm_model = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', LGBMRegressor(n_estimators=50, num_leaves=31, max_depth=10, random_state=42))])
lgbm_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.059242 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 974
[LightGBM] [Info] Number of data points in the train set: 283495, number of used features: 299
[LightGBM] [Info] Start training from score 4406.829461


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['RegistrationYear', 'Power', 'Mileage', 'RegistrationMonth'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'], dtype='object'))])),
                ('regressor',
                 LGBMRegressor(max_depth=10, n_estimators=50,
                               random_state=42))])

#Evaluación del rendimiento 📊

Evaluamos los modelos utilizando la métrica RMSE (Root Mean Squared Error), que mide el error medio entre las predicciones y los valores reales. Un menor RMSE indica una mayor precisión en las predicciones.  
En términos prácticos, un RMSE bajo significa que el modelo es capaz de predecir el precio de los coches usados con menos desviación respecto al precio real.


In [25]:
# Función para evaluar los modelos
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    print(f'RMSE: {rmse}')

# Evaluar los modelos
print("Linear Regression:")
evaluate_model(lr_model, X_test, y_test)

print("\nRandom Forest:")
evaluate_model(rf_model, X_test, y_test)

print("\nLightGBM:")
evaluate_model(lgbm_model, X_test, y_test)

Linear Regression:
RMSE: 3267.0482639050633

Random Forest:
RMSE: 2054.4823724485923

LightGBM:


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


RMSE: 1955.0911834846102


#Resultados de los modelos y análisis 📉
Los resultados obtenidos para cada modelo fueron los siguientes:


| Modelo              | RMSE (menor es mejor) |
|--------------------|----------------------|
| Regresión lineal    | 3267.05              |
| Random Forest       | 2054.48              |
| LightGBM            | 1955.09              |


**Conclusión:** El modelo LightGBM demostró ser el más preciso, seguido por Random Forest. La regresión lineal mostró un desempeño inferior debido a la naturaleza no lineal del problema.


# Conclusión

La lista de control parece estar correctamente marcada. Completado todas las etapas necesarias para el proyecto, desde la preparación de los datos hasta el entrenamiento y evaluación de los modelos. Ahora, vamos a elaborar una conclusión basándonos en el trabajo realizado y los resultados obtenidos.

En este proyecto, nos enfocamos en desarrollar un modelo predictivo para estimar el valor de mercado de coches usados para el servicio Rusty Bargain. Siguiendo las instrucciones del proyecto, realizamos la limpieza y el preprocesamiento de los datos, lo cual incluyó el manejo de valores faltantes y la eliminación de columnas no necesarias. Además, convertimos las características categóricas utilizando técnicas adecuadas para preparar nuestros datos para el modelado.

Para el modelado, probamos tres diferentes algoritmos: regresión lineal, bosque aleatorio (Random Forest) y LightGBM.
Estos modelos se seleccionaron para comparar un modelo lineal simple con modelos más complejos basados en árboles de decisión. Utilizamos el RMSE (Raíz del Error Cuadrático Medio) como métrica para evaluar el rendimiento de nuestros modelos.
Los resultados de RMSE fueron los siguientes:

- **Regresión lineal:** 3267.05
- **Random Forest:** 2054.48
- **LightGBM:** 1955.09

Basándonos en los resultados de RMSE, el modelo LightGBM demostró tener el mejor rendimiento, seguido por el Random Forest y, finalmente, la regresión lineal. Esto indica que los modelos basados en árboles de decisión y técnicas de potenciación del gradiente son más efectivos para este conjunto de datos en particular, lo cual es consistente con las expectativas, ya que estos modelos pueden capturar mejor las complejidades y las relaciones no lineales entre las características.

En conclusión, el modelo LightGBM, con un RMSE de 1955.09, parece ser la mejor opción para predecir el valor de mercado de los coches usados para Rusty Bargain. Este modelo no solo proporciona la predicción más precisa entre los evaluados, sino que también ofrece un buen equilibrio entre precisión y tiempo de entrenamiento, cumpliendo con los intereses de Rusty Bargain en cuanto a calidad de predicción, velocidad y eficiencia en el tiempo de entrenamiento.

### Posibles mejoras futuras:  
- Implementar una búsqueda exhaustiva de hiperparámetros (GridSearch o RandomizedSearch) para optimizar aún más el modelo.  
- Explorar técnicas adicionales de selección de características para identificar las variables más relevantes.  
- Incorporar datos adicionales como el historial de precios o variables externas (por ejemplo, la ubicación geográfica del vendedor).
